### Exercícios práticos 04 - Syanne Tavares

#### Resolução de sistemas lineares

- Podem ter 3 situações

1. Solução única -> retas concorrentes -> se tocam em 1 único ponto
2. Infinitas soluções -> retas coicidentes - pode ser qualquer tipo de valor
3. Sem solução -> Reta paralelas 

- 2 formas de resolver 
1. Métodos Diretos 
2. Métodos iterativos


## Exercício 01 - 

Implemente todas as técnicas de solução de sistemas lineares vistas em aula e faça uma comparação entre elas para as seguintes situações

# Métodos Diretos
## Fornecem soluções exatas com um número finito de operações. Podem ocorrer erros de arredondamento


# 1. Eliminação de Gauss
Resumidamente -> transforma o sistema em um sistema equivalente do tipo triangular superior e resolve a partir disso

In [2]:
import numpy as np
import pandas as pd

  
# Resolve o sistema triangular superior.
def SistemaTriangularSuperior(Ax,b):
  """Função que resolve um sistema linear triangular superior do tipo  𝐴𝑥 = b que recebe como parâmetro """
  n = len(b) #tamanho do vetor b
  x= [b[i]/Ax[i][i] if i == 0 else None for i in range(0,len(b))]

  x[n-1] = b[n-1]/Ax[n-1][n-1]

  for i in list(range(n-1,0,-1)):
    s = 0
    for j in list(range(i+1,n+1)):
      s = s + Ax[i-1][j-1]*x[j-1]

    x[i-1] = (b[i-1]-s)/(Ax[i-1][i-1])

  return x

def EliminacaoDeGauss(A,b):
  """Função que resolve um sistema linear do tipo Ax = b que recebe como parâmetro
     Return : retorna o vetor x resultante"""
  
  # ELIMINAÇÃO DE GAUSS
  n = len(b)

  # Calculo dos pivos.
  for k in range(1,n+1):
    # PIVOTEAMENTO PARCIAL
    for i in range(k+1, len(A)+1):
      if (abs(A[i-1][k-1]) > abs(A[k-1][k-1])):
        [A[k-1], A[i-1]] = [A[i-1], A[k-1]]
        [b[k-1], b[i-1]] = [b[i-1], b[k-1]]

    # Calculo dos multiplicadores.
    for i in list(range(k+1,n+1)):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = 0
      
      # Atualizar demais valores da linha
      for j in list(range(k+1,n+1)):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1]
      b[i-1] = b[i-1] - m*b[k-1]

  # print("depois da eliminação\n Ax =  ",Ax2,"\nb = ",b2)
  x = SistemaTriangularSuperior(A,b)
  return(x)


In [3]:
Ax = [[3, 2, 4],
      [1, 1, 2],
      [4, 3,-2]]
b = [1, 2, 3]

print("X: ",EliminacaoDeGauss(Ax,b))

X:  [-3.0, 5.0, 0.0]


In [4]:
#exemplo 2
Ax = [[0,9, 8],
    [4, 5 ,6],
    [6, 4, 9]]

b =  [105, 83 ,93]
print("X: ",EliminacaoDeGauss(Ax,b))

X:  [4.999999999999999, 9.0, 3.000000000000001]


# Fatoração LU com pivoteamento parcial

#### O processo de fatoração consistem em decompor a matriz A em um produto de dois ou mais fatores
▪ De forma que seja mais fácil resolver o sistema em etapas

In [5]:
def formata_matriz(M):
  """Função que printa uma matriz formatando-a"""
  m = len(M) # número de linhas
  n = len(M[0]) # número de colunas
  s = ""
  for i in range(m):
      for j in range(n):
          s += "%9.3f " % M[i][j]
      s +=  "\n"
  return s

# Resolve o sistema triangular inferior.
def SistemaTriangularInferior(Ax,b):
  """Função que resolve um sistema triangular inferior"""
  n = len(b)
  
  y= [b[i]/Ax[i][i] if i == 0 else None for i in range(0,len(b))]

  for i in list(range(1,n+1,1)):
    s = 0
    for j in list(range(1,i,1)):
      s = s + Ax[i-1][j-1]*y[j-1]

    y[i-1] = b[i-1] - s

  return y

def LU_PivParc(A,b):
  """Função que resolve um sistema linear do tipo Ax = b usando o método de Fatoração LU com pivoteamento parcial 
     A = matriz de coeficientes
     b = vetor b resultante
     Retorna um vetor x calculado"""
  n = len(A)
  p=[0]*n
  

  ###### pivoteamento parcial 
  p = list(range(1,n+1)) #"matriz permutação"

  for k in range(1,n):
    pv = abs(A[k-1][k-1])
    Lpivo = k

    for i in range(k+1,n+1):
        if abs(A[i-1][k-1])> pv :
          pv = abs(A[i-1][k-1])
          Lpivo = i
  
    if pv == 0: 
      return None #matriz singular
      

    if Lpivo != k : # se for diferente então tem que realizar a permutação
      [p[k-1],p[Lpivo-1]] = [p[Lpivo-1],p[k-1]] # recurso guanabaristico
    
      for j in range(1,n+1):
          [A[k-1][j-1],A[Lpivo-1][j-1]] = [A[Lpivo-1][j-1],A[k-1][j-1]] 

  # resolução dos sistemas triangulares
  # Calculo dos pivos.
  for k in list(range(1,n,1)):
    # Calculo dos multiplicadores.
    for i in list(range(k+1,n+1,1)):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = m
      # Atualizar demais valores da linha
      for j in list(range(k+1,n+1,1)):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1]
  ### multiplicação da b pela "matriz permutação"
  c=[None for _ in range(0,len(b))]
  for i in range(1,n+1):
    Lpivo = p[i-1]
    c[i-1]=b[Lpivo-1]

  # resolução dos sistemas 
  y = SistemaTriangularInferior(A,c)
  x = SistemaTriangularSuperior(A,y)

  return x

In [6]:
# Teste com dados.
Ai = [[3, -4, 1],
      [1, 2, 2],
      [4, 0, -3]]

bi = [9, 3, -2]

print("Ai = \n",formata_matriz(Ai))
#Obtendo os fatores L e U.
print("X =")
print(LU_PivParc(Ai,bi))

Ai = 
     3.000    -4.000     1.000 
    1.000     2.000     2.000 
    4.000     0.000    -3.000 

X =
[1.0, -1.0, 2.0]


## Métodos iterativos

### 3. Gauss-Jacobi

In [7]:
import math
def TesteConvergencia(MatrizA):
    """Função que realiza teste de convergência pra uma matriz A
        Retorna True caso a seja convergente e False caso contrário"""
    Alfas=list()

    for i in range(len(MatrizA)):
        s=0
        for j in range(len(MatrizA)):
            if j!=i:
                s+=MatrizA[i][j]

        Alfas.append(s/MatrizA[i][i])
    # print(Alfas)
    # print(max(Alfas))

    if max(Alfas) < 1:
        return True
    else:
        return False
def criterioParada(x,xk,precisao):
    """Função que compara x com x-1 verificando a precisão do método como um critério de parada"""
    soma = 0
    zip_object = zip(x, xk) # une as informações de duas variáveis
    for list1_i, list2_i in zip_object:
        soma = soma + math.fabs(list1_i-list2_i)

    if (soma < precisao):
        return True
    else:
        return False   
            
def GaussJacobi(MatrizA,b,max_iteracoes,precisao):
    """Função que resolve um sistema linear do tipo Ax = b através do método iterativo de Gauss-Jacobi 
    Parâmetros:
        MatrizA = matriz de coeficientes do sistema linear
        b = vetor b
        max_iteracoes = npumero máximo de iterações
        precisao = precisão do erro
        
        retorno: tupla com
      vetor x resultante e com o número de iterações"""
    n= len(b) #tamanho de do vetor base

    X=b.copy() # copiando b pra x como chute inicial
    solucao = True
    # gerar chute inicial
    for i in list(range(1,n+1)): # percorre cada elemento de x de 1 até n
        # dividir pelo elemento da diagonal principal
        if math.fabs(MatrizA[i-1][i-1])> 0 :# verifica se é possível realizar divisão se a diagonal é diferente de zero
            X[i-1] = b[i-1]/MatrizA[i-1][i-1]
        else:
            solucao = False # não se pode calcular o chute inicial
            break
    if solucao and TesteConvergencia(MatrizA):
        # print("Iteração 0")
        # print("x = ",X)

        xk = X.copy()

        iter = 0
        
        # realiza o calculo de uma função iteração
        while (iter < max_iteracoes): # início das iterações
            iter = iter + 1
            for i in list(range(1,n+1)):
                s = 0

                #realiza o somatório
                for j in list(range(1,n+1)):
                    if ((i-1) != (j-1)):
                        s = s + MatrizA[i-1][j-1]*X[j-1] # somando os termos que não são da diagonal

                xk[i-1] = (1/MatrizA[i-1][i-1])*(b[i-1]-s) # equivale a 1/a(ii)*(bi - somatório de aij*xj)
                
            # print("Iteração: ",iter)
            # print("xk = ",xk)

            #verifica o critério de parada ->  O processo é repetido até ue x^k e x^k-1 eestejam suficientemente próximos
            if criterioParada(X,xk,precisao):
                X = xk.copy()
                break    
            X = xk.copy()
      
        return X,iter
    else:
        return None,None


In [8]:
#exemplo 

A = [[10, 2,  1],
     [ 1, 5,  1],
     [ 2, 3, 10]]

b = [7, -8, 6]

x,iter = GaussJacobi(A,b,10,0.01)
print("x = ",x)
print("N iterações = ",iter)



x =  [1.000236, -1.9989360000000003, 1.000284]
N iterações =  5


### 4. Gauss-Seidel

In [9]:
def GaussSeidel(MatrizA,b,max_iteracoes,precisao):
    """Função que resolve um sistema linear do tipo Ax = b através do método iterativo de Gauss-Seidel
      Parâmetros:
      MatrizA = matriz de coeficientes do sistema linear
      b = vetor b
      max_iteracoes = npumero máximo de iterações
      precisao = precisão do erro
      
      retorno: tupla com
      vetor x resultante e com o número de iterações """
    n= len(b) #tamanho de do vetor base

    X=b.copy() # copiando b pra x como chute inicial
    solucao = True
    # gerar chute inicial
    for i in list(range(1,n+1)): # percorre cada elemento de x de 1 até n
        # dividir pelo elemento da diagonal principal
        if math.fabs(MatrizA[i-1][i-1])> 0 :# verifica se é possível realizar divisão se a diagonal é diferente de zero
            X[i-1] = b[i-1]/MatrizA[i-1][i-1]
        else:
            solucao = False # não se pode calcular o chute inicial
            break
    if solucao and TesteConvergencia(MatrizA):
      # print("Iteração 0")
      # print("x = ",X)

      xk = X.copy()

      iter    = 0
      
      # realiza o calculo de uma função iteração
      while (iter < max_iteracoes): # início das iterações
        iter = iter + 1
        for i in list(range(1,n+1)):
          s = 0

          #realiza o somatório
          for j in list(range(1,n+1)):
            if ((i-1) > (j-1)): # se o índice da linha é maior que o da coluna
              s = s + MatrizA[i-1][j-1]*xk[j-1] # usa o xk que tem os valores atualizados
            elif ((i-1) < (j-1)): # se o índice da linha é maior que o da coluna
              s = s + MatrizA[i-1][j-1]*X[j-1] 

          xk[i-1] = (1/MatrizA[i-1][i-1])*(b[i-1]-s) # equivale a 1/a(ii)*(bi - somatório de aij*xj)
      
        # print("Iteração: ",iter)
        # print("xk = ",xk)

        #verifica o critério de parada ->  O processo é repetido até ue x^k e x^k-1 eestejam suficientemente próximos
        if criterioParada(X,xk,precisao):
          X = xk.copy()
          break    
        X = xk.copy()
     
      return X,iter
    else:
       return None,None

In [10]:
#exemplo 

A = [[10, 2,  1],
     [ 1, 5,  1],
     [ 2, 3, 10]]

b = [7, -8, 6]

x,iter = GaussSeidel(A,b,10,0.01)
print("x = ",x)
print("N iterações = ",iter)


x =  [0.9999577430400001, -2.000046123328, 1.0000222883904002]
N iterações =  4


## Comparação entre os métodos

In [11]:
def GeraMatriz(n):
    """Função que gera matriz quadrada de simensão nxn com inteiros aleatórios """

    array = np.random.randint(10, size=(n, n))

    return array

def MultiplicaMatrizes(MatrizA,MatrizB):
    """Função que multiplica duas matrizes que recebe com parâmatro e retorna o resultado"""

    result=np.zeros((len(MatrizA),len(MatrizA)))
  
    result = np.dot(MatrizA,MatrizB) 
    
    # for r in result: 
    #     print(r)
    return result 

In [12]:
# exemplo 

#n = 10
b  = np.random.randint(10, size=(10)) 
Matriz_10 = GeraMatriz(10)

print(Matriz_10)
print("b = " ,b)

MultiplicaMatrizes(Matriz_10,b)



[[2 3 4 6 0 4 4 5 6 2]
 [5 8 3 4 2 3 6 3 2 1]
 [7 1 5 0 4 6 2 0 4 5]
 [5 3 8 7 2 4 5 7 5 1]
 [9 4 5 0 1 9 3 7 7 1]
 [7 3 8 2 7 5 4 2 6 3]
 [4 7 3 2 5 7 1 2 7 9]
 [2 2 3 2 8 4 7 6 4 0]
 [3 9 9 9 2 3 2 3 9 9]
 [0 4 2 3 9 6 7 2 5 9]]
b =  [7 0 4 1 7 0 6 9 0 1]


array([107, 129, 114, 182, 172, 177, 110, 180, 128, 143])

In [13]:
def GetExemplo(Dimensao_n):
    x = np.random.randint(10, size=(Dimensao_n)) 
    Matriz = GeraMatriz(Dimensao_n)
    b = MultiplicaMatrizes(Matriz,x)

    print(f"Matriz{Dimensao_n}x{Dimensao_n} Gerada :")
    print(Matriz)
    print("x = " ,x)
    print("b = ",b)
    print("Ax = b")
    
    return Matriz,x,b

### Exemplo 1 -> n = 10

In [14]:
Ax = [[0,9, 8],
    [4, 5 ,6],
    [6, 4, 9]]

b =  [105, 83 ,93]
print("X: ",EliminacaoDeGauss(Ax,b))

X:  [4.999999999999999, 9.0, 3.000000000000001]


In [15]:
Ax = [[0,9, 8],
    [4, 5 ,6],
    [6, 4, 9]]

b =  [105, 83 ,93]
print("X: ",LU_PivParc(Ax,b))

X:  [4.999999999999997, 8.999999999999996, 3.0000000000000027]


In [16]:
Ax = [[0,9, 8],
    [4, 5 ,6],
    [6, 4, 9]]

b =  [105, 83 ,93]
resultJacobi,iteracoes = GaussJacobi(Ax,b,10000,0.000001)
print(f"X por Gauss-Jacobi: {resultJacobi}, com {iteracoes} iterações")

X por Gauss-Jacobi: None, com None iterações


In [28]:
Ax,x,b = GetExemplo(4)
print("X por eliminação de gauss: ",EliminacaoDeGauss(Ax,b))
print("X por Fatoração LU com pivoteamento parcial: ",LU_PivParc(Ax,b))
resultJacobi,iteracoes = GaussJacobi(Ax,b,100,0.000001)
print(f"X por Gauss-Jacobi: {resultJacobi}, com {iteracoes} iterações")
resultSeidel,iteracoes = GaussSeidel(Ax,b,1000,0.000001)
print(f"X por Gauss-Seidel: {resultSeidel}, com {iteracoes} iterações")

Matriz4x4 Gerada :
[[4 0 4 1]
 [0 4 4 0]
 [0 0 8 9]
 [3 0 8 5]]
x =  [4 3 7 7]
b =  [ 51  40 119 103]
Ax = b
X por eliminação de gauss:  [6.625, 6.375, 3.625, 10.0]
X por Fatoração LU com pivoteamento parcial:  [6.625, 6.375, 3.625, 10.0]
X por Gauss-Jacobi: None, com None iterações
X por Gauss-Seidel: None, com None iterações
